In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer
from collections import defaultdict
import string
import random 
import re

In [2]:
from sklearn.svm import LinearSVC,SVC
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, SGDClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [3]:
def preprocess_text_tags(text):
    if isinstance(text,(int,float)):
        return ''
    text = text.lower()
    text = re.sub(r'[^аәбвгғдеёжзийкқлмнңоөпрстуұүфхһцчшщъыіьэюя]', ' ', text)
    text = re.sub(' +',' ', text)
    return ' '.join(text.split()) 

In [4]:
rudata=None
with open('ru.txt','r',encoding="utf-8") as file:
    rudata=file.read().replace('\n',' ')
rudata=rudata.replace('\xa0',' ')
rudata=rudata.replace('\xad',' ')


In [5]:
kzdata=None
with open('kz.txt','r',encoding="utf-8") as file:
    kzdata=file.read().replace('\n',' ')
kzdata=kzdata.replace('\xa0',' ')
kzdata=kzdata.replace('\xad',' ')

In [6]:
kzdata[:1000]

'15 қазанда Алматы облысының Нарынқол ауылында туған. Абай атындағы Қазақ педагогикалық институтын, кейіннен Мәскеуде Жоғары әдеби курсты бітірген. Біраз жыл ауыл мектебінде бала оқытқан. «Қазақ әдебиеті» газетінде, «Балдырған» журналында, Ш.Айманов атындағы «Қазақфильм» киностудиясында істеген. Қазақстан Жазушылар одағында балалар әдебиеті жөніндегі әдеби кеңесші болған.  Шығармашылық жолын өлеңмен бастап, 1950 жылы «Бұлақ» атты жыр жинағын шығарған. Кейін балалар мен жасөспірімдерге арналған 20-дан астам әңгіме, повесть, роман кітаптарын ұсынды. «Менің атым Қожа», «Балалық шаққа саяхат», «Бозтөбеде бір қыз бар», «Өлгендер қайтып келмейді» және «Қайдасың, Гауһар» секілді туындылары КСРО халықтарының және шетелдердің көптеген тілдеріне аударылып, сахналық, экрандық нұсқаға айналған. 1967 жылы Балалар мен жасөспірімдерге арналған фильмдердің Канн қаласында (Франция) өткен халықаралық фестивалінде «Менің атым Қожа» фильмі (Б.Соқпақбаевтың сценарийі) арнаулы жүлдеге ие болды.  Қазақ КСР Ж

In [7]:
rudata[:1000]

'### Глава первая В кроличьей норе  Алисе надоело сидеть на пригорке рядом с сестрой и ничего не делать. Раза два она заглянула украдкой в книгу, которую читала её сестра, но там не было ни разговоров, ни картинок. «Какой толк в книге, -- подумала Алиса, -- если в ней нет ни картинок, ни разговоров?»  Потом она стала раздумывать (насколько вообще это возможно в такой невыносимо жаркий день, когда одолевает дремота), сто́ит ли ей вставать, чтобы пойти нарвать маргаритки и сплести венок, или нет, как вдруг Белый Кролик с розовыми глазками пробежал мимо неё.  В этом не было, конечно, ничего особенного. Не удивилась Алиса и тогда, когда Кролик пробормотал себе под нос:  -- Ах, боже мой, я опоздаю!  Думая об этом впоследствии, Алиса не могла понять, почему же она не удивилась, услышав, что Кролик заговорил, но в тот момент это не показалось ей странным. Однако, когда Кролик вынул из жилетного кармана часы и, взглянув на них, побежал дальше, Алиса вскочила, сообразив, что никогда ещё не случ

In [8]:
rusentences=re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s", rudata)

In [9]:
kzsentences=re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)\s", kzdata)

In [10]:
len(rusentences)

95884

In [11]:
len(kzsentences)

112724

In [12]:
kzsentences[:10]

['15 қазанда Алматы облысының Нарынқол ауылында туған.',
 'Абай атындағы Қазақ педагогикалық институтын, кейіннен Мәскеуде Жоғары әдеби курсты бітірген.',
 'Біраз жыл ауыл мектебінде бала оқытқан.',
 '«Қазақ әдебиеті» газетінде, «Балдырған» журналында, Ш.Айманов атындағы «Қазақфильм» киностудиясында істеген.',
 'Қазақстан Жазушылар одағында балалар әдебиеті жөніндегі әдеби кеңесші болған.',
 ' Шығармашылық жолын өлеңмен бастап, 1950 жылы «Бұлақ» атты жыр жинағын шығарған.',
 'Кейін балалар мен жасөспірімдерге арналған 20-дан астам әңгіме, повесть, роман кітаптарын ұсынды.',
 '«Менің атым Қожа», «Балалық шаққа саяхат», «Бозтөбеде бір қыз бар», «Өлгендер қайтып келмейді» және «Қайдасың, Гауһар» секілді туындылары КСРО халықтарының және шетелдердің көптеген тілдеріне аударылып, сахналық, экрандық нұсқаға айналған.',
 '1967 жылы Балалар мен жасөспірімдерге арналған фильмдердің Канн қаласында (Франция) өткен халықаралық фестивалінде «Менің атым Қожа» фильмі (Б.Соқпақбаевтың сценарийі) арнау

In [13]:
rusentences[:10]

['### Глава первая В кроличьей норе  Алисе надоело сидеть на пригорке рядом с сестрой и ничего не делать.',
 'Раза два она заглянула украдкой в книгу, которую читала её сестра, но там не было ни разговоров, ни картинок.',
 '«Какой толк в книге, -- подумала Алиса, -- если в ней нет ни картинок, ни разговоров?»  Потом она стала раздумывать (насколько вообще это возможно в такой невыносимо жаркий день, когда одолевает дремота), сто́ит ли ей вставать, чтобы пойти нарвать маргаритки и сплести венок, или нет, как вдруг Белый Кролик с розовыми глазками пробежал мимо неё.',
 ' В этом не было, конечно, ничего особенного.',
 'Не удивилась Алиса и тогда, когда Кролик пробормотал себе под нос:  -- Ах, боже мой, я опоздаю!',
 ' Думая об этом впоследствии, Алиса не могла понять, почему же она не удивилась, услышав, что Кролик заговорил, но в тот момент это не показалось ей странным.',
 'Однако, когда Кролик вынул из жилетного кармана часы и, взглянув на них, побежал дальше, Алиса вскочила, сообразив

In [14]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

In [15]:
df_kz=pd.DataFrame(kzsentences)

In [16]:
df_ru=pd.DataFrame(rusentences)

In [17]:
df_kz['language']=0

In [18]:
df_ru['language']=1

In [19]:
df=pd.concat([df_kz,df_ru])

In [20]:
df.rename(columns = {0: 'Text'}, inplace = True)

In [21]:
display(df)

,Text,language
0,15 қазанда Алматы облысының Нарынқол ауылында ...,0
1,"Абай атындағы Қазақ педагогикалық институтын, ...",0
2,Біраз жыл ауыл мектебінде бала оқытқан.,0
3,"«Қазақ әдебиеті» газетінде, «Балдырған» журнал...",0
4,Қазақстан Жазушылар одағында балалар әдебиеті ...,0
...,...,...
95879,*1938* ::: [^1]: Ипсон -- местечко близ Лонд...,1
95880,"[^2]: Минотавр -- мифическое чудовище, полубы...",1
95881,"Минотавр питался мясом преступников, а так...",1
95882,Убит был Минотавр Тезеем -- мифическим афински...,1


In [22]:
df['clean_text'] = df['Text'].map(preprocess_text_tags)

In [23]:
df["language"].value_counts()

0    112724
1     95884
Name: language, dtype: int64

In [24]:
X = df["clean_text"]
y = df["language"]

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [26]:
from sklearn.metrics import accuracy_score, roc_curve, auc
from sklearn import metrics

tfidf = TfidfVectorizer(
       ngram_range=(1, 3),
       analyzer='char_wb',
       use_idf=1,
       smooth_idf=1)

data_train_count = tfidf.fit_transform(x_train)
data_test_count  = tfidf.transform(x_test)
clf = MultinomialNB()
clf.fit(data_train_count, y_train)
pred = clf.predict(data_test_count)
print(metrics.classification_report(y_test, pred, digits=3)) 

              precision    recall  f1-score   support

           0      0.978     0.994     0.986     22631
           1      0.992     0.973     0.983     19091

    accuracy                          0.984     41722
   macro avg      0.985     0.983     0.984     41722
weighted avg      0.984     0.984     0.984     41722



In [27]:
def label_lang(row):
    mixed_text=row['clear_text']  
    if not mixed_text or len(mixed_text)<2:
        return 1
    data_test_count  = tfidf.transform([mixed_text])
    pred = clf.predict(data_test_count)
    return pred[0]

In [32]:
import pickle
pickle.dump(tfidf, open('tfidf_model.sav', 'wb'))
pickle.dump(clf, open('clf_model.sav', 'wb'))